In [11]:
from glob import glob
import subprocess
import os 
from nilearn.image import resample_to_img
import nibabel as nib
from tqdm import tqdm
from nimlab import datasets as nimds


from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from calvin_utils.file_utils.import_functions import GiiNiiFileImport


def downsample_image(input_path, output_path):
    """
    Function to downsample a 3D image to a new voxel size using a target affine.
    
    Args:
    input_path (str): Filepath to the input image.
    output_path (str): Filepath to save the output image.
    target_voxel_size (list): Target voxels to resample to.
    """
    # Load the image
    img = nib.load(input_path)
    mni_mask = nimds.get_img("mni_icbm152")
    
    # Downsample the image using the target affine
    resampled_img = resample_to_img(img, mni_mask)

    # Save the downsampled image
    nib.save(resampled_img, output_path)
# User-defined variables
root_dir = r"/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs"
file_pattern_1 = r"/stimulations/MNI152NLin2009bAsym/gs_2023Aysu/*sim-binary_model-simbio_hemi-R.nii"
file_pattern_2 = r"/stimulations/MNI152NLin2009bAsym/gs_2023Aysu/*sim-binary_model-simbio_hemi-L.nii"
output_file_pattern = r"sim-efield_model-simbio_hemi-bl.nii"

# Path to MNI template
mni_template_path = r"/Users/cu135/Dropbox (Partners HealthCare)/resources/mni_spaces/mni_icbm152_nlin_sym_09b_nifti/mni_icbm152_nlin_sym_09b/mni_icbm152_t1_tal_nlin_sym_09b_hires.nii"

for sub_dir in tqdm(os.listdir(root_dir)):
    sub_dir_path = os.path.join(root_dir, sub_dir)
    if not os.path.isdir(sub_dir_path):
        continue
    # print("Subdirpath:", os.path.join(root_dir, sub_dir))
    files1 = glob(root_dir+"/"+sub_dir+file_pattern_1)
    files2 = glob(root_dir+"/"+sub_dir+file_pattern_2)

    if files1 and files2:
        # Extract identified files
        file1 = files1[0]
        file2 = files2[0]
        
        # Prepare Output File
        output_dir = os.path.dirname(file1)
        output_file_with_subject = os.path.basename(sub_dir_path)+"_"+output_file_pattern
        
        # Prepare backup file name
        copyfile1 = file1.split('.nii')[0] + "_copy.nii"
        copyfile2 = file2.split('.nii')[0] + "_copy.nii"
        
        # Copy files to create backups
        # shutil.copyfile(file1, copyfile1)
        # shutil.copyfile(file2, copyfile2)
        
        # Identify the resampled file name
        resamp1 = copyfile1.split('.nii')[0]+"_resampled.nii"
        resamp2 = copyfile2.split('.nii')[0]+"_resampled.nii"
        
        # Attempt resampling to MNI space to fix bad headers
        subprocess.run(["flirt", "-in", copyfile1, "-ref", mni_template_path, "-out", resamp1], check=True)
        subprocess.run(["flirt", "-in", copyfile2, "-ref", mni_template_path, "-out", resamp2], check=True)
        
        # Identify the downsampled file name
        downfile1 = resamp1.split('.nii')[0]+"_mni152.nii"
        downfile2 = resamp2.split('.nii')[0]+"_mni152.nii"
        
        # Add the aligned files
        downsample_image(resamp1, downfile1)
        downsample_image(resamp2, downfile2)
        
        try:
            importer = GiiNiiFileImport(os.path.dirname(downfile1), file_pattern="*sim-binary_model-simbio_hemi-*_mni152*.nii")
            imports = importer.run()
            imports["summated"] = imports.iloc[:,0] + imports.iloc[:,1]
            display(imports["summated"].to_numpy())
            
            nifti_from_matrix(matrix=imports["summated"].to_numpy(), output_file=output_dir, output_name=output_file_with_subject, reference="MNI")
            
            # Clean up and remove the copy files. 
            os.remove(copyfile1)
            os.remove(copyfile2)
            os.remove(resamp1)
            os.remove(resamp2)            
            os.remove(downfile1)
            os.remove(downfile2)
            
            print(f"\n ***Processed: {sub_dir_path}")
            
        except subprocess.CalledProcessError as e:
            print(f"Error processing {sub_dir_path}: {e}")
    else:
        print(f"Missing file(s) in {sub_dir_path}")

  0%|          | 0/61 [00:00<?, ?it/s]

Attempting to import from: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS034/stimulations/MNI152NLin2009bAsym/gs_2023Aysu/*sim-binary_model-simbio_hemi-*_mni152*.nii


array([0., 0., 0., ..., 0., 0., 0.])

  3%|▎         | 2/61 [00:00<00:07,  8.39it/s]

Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS034/stimulations/MNI152NLin2009bAsym/gs_2023Aysu

 ***Processed: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS034
Attempting to import from: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS060/stimulations/MNI152NLin2009bAsym/gs_2023Aysu/*sim-binary_model-simbio_hemi-*_mni152*.nii


...

 98%|█████████▊| 60/61 [00:15<00:00,  4.04it/s]

Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS046/stimulations/MNI152NLin2009bAsym/gs_2023Aysu

 ***Processed: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS046
Attempting to import from: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS079/stimulations/MNI152NLin2009bAsym/gs_2023Aysu/*sim-binary_model-simbio_hemi-*_mni152*.nii


array([0., 0., 0., ..., 0., 0., 0.])

100%|██████████| 61/61 [00:15<00:00,  3.96it/s]

Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS079/stimulations/MNI152NLin2009bAsym/gs_2023Aysu

 ***Processed: /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS079
